### Creación de gpkg de division

In [ ]:
from utils_pre_procesamiento import generar_tiles_spatial_split

tif_sierra = r""
out_dir_sierra = r""

tif_selva = r""
out_dir_selva = r""

generar_tiles_spatial_split(tif_sierra, out_dir = out_dir_sierra)
generar_tiles_spatial_split(tif_selva, out_dir = out_dir_selva)

Píxeles válidos totales: 588882000
Val   asignado: 117781300 (20.0%)
Test asignado: 70667800 (12.0%)
Train: 400432900
Píxeles válidos totales: 591685632
Val   asignado: 141992813 (24.0%)
Test asignado: 70996458 (12.0%)
Train: 378696361


### Creacion de parches

In [ ]:
from raster_to_tile import data_split

tif_selva = r""
tif_sierra = r""

mask_casas_selva_path = r""
mask_casas_sierra_path = r""

mask_caminos_selva_path = r""
mask_caminos_sierra_path = r""

gpkg_division_selva_tile = r""
gpkg_division_sierra_tile = r""

dataset_general_out = r""

### Area Excluida
gpkg_area_ciudad = r""

#Sierra
data_split(tif_sierra, mask_casas_sierra_path, mask_caminos_sierra_path, gpkg_division_sierra_tile, dataset_general_out, gpkg_area_excluida_path=gpkg_area_ciudad)
#Selva
data_split(tif_selva, mask_casas_selva_path, mask_caminos_selva_path, gpkg_division_selva_tile, dataset_general_out)

Leyendo raster entero para calcular percentiles para posterior normalización de parches
Cargando raster...
Cargando geometrías...
Geometria de : Casas
Geometria de : Caminos
Geometria de : Area fuera de interes
Cargando tiles...

Procesando tile 0 → train

Procesando tile 1 → train

Procesando tile 2 → val

Procesando tile 3 → train

Procesando tile 4 → train

Procesando tile 5 → train

Procesando tile 6 → train

Procesando tile 7 → train

Procesando tile 8 → test

Procesando tile 9 → train

Procesando tile 10 → val

Procesando tile 11 → train

Procesando tile 12 → val

Procesando tile 13 → train

Procesando tile 14 → train

Procesando tile 15 → train

Procesando tile 16 → train

Procesando tile 17 → train

Procesando tile 18 → test

Procesando tile 19 → train

Procesando tile 20 → val

Procesando tile 21 → test

Procesando tile 22 → val

Procesando tile 23 → train

Procesando tile 24 → train

 LISTO: Dataset generado correctamente.
Total de parches generados: 002705_006024
Leyendo ras

### Verificar GPU disponible

In [ ]:
import tensorflow as tf
print("TF version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs:", tf.config.list_physical_devices('GPU'))

In [ ]:
from utils2 import split_dataset

(train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks) = split_dataset(carpeta=dataset_general_out)

In [ ]:
from TiffDataGeneratorAugmentedUnet import TIFFDataGeneratorAug

# Generadores
train_gen = TIFFDataGeneratorAug(train_imgs, train_masks, normalize='imagenet',batch_size=1, n_channels=3, augment=True)#<--- canales
val_gen = TIFFDataGeneratorAug(val_imgs, val_masks, normalize='imagenet', batch_size=1, shuffle=False, n_channels=3, augment=False)#<--- canales
test_gen = TIFFDataGeneratorAug(test_imgs, test_masks, normalize='imagenet', batch_size=1, shuffle=False, n_channels=3, augment=False)#<--- canales


In [18]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

In [ ]:
from tensorflow.keras.optimizers import Adam
import configuraciones as cfg
import tensorflow as tf


BACKBONE = 'resnet34'#efficientnetb0 tmb puede ser (buena con imag satelitales)
model = sm.Unet(
    backbone_name=BACKBONE,
    encoder_weights='imagenet',
    classes=3,
    activation='softmax'
    #input_shape=(512, 512, 4)
)

# Definir nombres de clases
class_names = ['background', 'road', 'building']

model.compile(
    optimizer=Adam(1e-4),
    loss=cfg.masked_sparse_cce,
    metrics=[
        cfg.MaskedSparseCategoricalAccuracy(num_classes=3, class_names=class_names, name='acc'),
        cfg.MaskedIoUPerClass(num_classes=3, class_names=class_names, name='miou')
    ]
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

early_stop = EarlyStopping(
    monitor='val_loss',      # métrica que se evalúa
    mode='min',
    patience=10,
    restore_best_weights=True,
    verbose= 1
)

checkpoint = ModelCheckpoint(
    r"checkpoint.keras",         # ruta del archivo
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau( # reduccion lr auto
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

In [ ]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    verbose=1,
    callbacks=[early_stop, checkpoint, reduce_lr, cfg.PrintMetricsPerClass()]
)
print("Epochs realizadas:", len(history.history['loss']))

In [ ]:
# Después del entrenamiento, obtener IoU por clase
iou_metric = model.metrics[-1]  # Última métrica
iou_per_class = iou_metric.get_iou_per_class()

print("\nIoU por clase:")
for class_name, iou_value in iou_per_class.items():
    print(f"  {class_name}: {iou_value:.4f}")

In [ ]:
# Save final model
model.save("model_final.keras")

# Evaluate on train set
print("\n=== Evaluación en test set ===")
train_results = model.evaluate(train_gen, verbose=1)
print(f"Train Loss: {train_results[0]:.4f}")
print(f"Train Masked Accuracy: {train_results[1]:.4f}")
print(f"Train Masked Mean IoU: {train_results[2]:.4f}")
# Evaluate on test set
print("\n=== Evaluación en val set ===")
val_results = model.evaluate(val_gen, verbose=1)
print(f"Val Loss: {val_results[0]:.4f}")
print(f"Val Masked Accuracy: {val_results[1]:.4f}")
print(f"Val Masked Mean IoU: {val_results[2]:.4f}")
# Evaluate on test set
print("\n=== Evaluación en test set ===")
test_results = model.evaluate(test_gen, verbose=1)
print(f"Test Loss: {test_results[0]:.4f}")
print(f"Test Masked Accuracy: {test_results[1]:.4f}")
print(f"Test Masked Mean IoU: {test_results[2]:.4f}")